# Importing packages

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten, BatchNormalization,Dropout,Activation
import matplotlib.pyplot as plt
from keras import callbacks
import numpy as np
from keras.applications.vgg16 import VGG16
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,ConfusionMatrixDisplay
import seaborn as sns
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

# Data Preprocessing / Data Augmentation

In [7]:
train_data_path = '/content/drive/MyDrive/Colab Notebooks/FED_DATASET/train'
train_data_path = '/content/drive/Othercomputers/My Laptop/My Data/MS HIS/3 Winter Semester 2024-2025/RT/Dataset/train'

test_data_path = '/content/drive/MyDrive/Colab Notebooks/FED_DATASET/test'
test_data_path = '/content/drive/Othercomputers/My Laptop/My Data/MS HIS/3 Winter Semester 2024-2025/RT/Dataset/test'

model_weights = '/content/drive/MyDrive/Colab Notebooks/model.weights.h5'
model_weights = '/content/drive/Othercomputers/My Laptop/My Data/MS HIS/3 Winter Semester 2024-2025/RT/weights/model.weights.h5'

model_save_path = '/content/drive/Othercomputers/My Laptop/My Data/MS HIS/3 Winter Semester 2024-2025/RT/models/model.json'

In [3]:
batch_size=32
img_size=48
# Data Augmentation
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    shear_range=0.3,
                    zoom_range=0.3,
                    width_shift_range = 0.1,
                    height_shift_range = 0.1,
                    horizontal_flip=True,
                    validation_split=0.3)

validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

train_set = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(img_size, img_size),
        color_mode =  'grayscale',
        batch_size=batch_size,
        class_mode='categorical',
        subset = 'training')

validation_set = validation_datagen.flow_from_directory(
        test_data_path,
        target_size=(img_size, img_size),
        color_mode =  'grayscale',
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')


Found 20099 images belonging to 7 classes.
Found 2151 images belonging to 7 classes.


# CNN Architecture

In [ ]:
from keras.optimizers import Adam,SGD,RMSprop

no_of_classes = 7

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same', activation='relu',
                 input_shape = (48,48,1) ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))

model.summary()

# Compiling the Model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Model Training

In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

epochs = 20

checkpoint = ModelCheckpoint(model_weights, monitor='val_acc', verbose=1, save_weights_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')

callbacks = [checkpoint,reduce_learningrate]


history = model.fit(train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=epochs,
                                validation_data = validation_set ,
                                validation_steps = validation_set.n//validation_set.batch_size,
                                callbacks=callbacks
                                )

In [ ]:
# Converting the model into JSON format and storing it in "model.json" file.
model_json = model.to_json()
with open(model_save_path, "w") as json_file:
    json_file.write(model_json)

# Comparing loss

In [ ]:
# Comparing loss
plt.plot(history.history['loss'], label='Training Loss', color='red')
plt.plot(history.history['val_loss'], label='Validation Loss', color='blue')
plt.legend()
plt.show()

# Comparing Accuracy

In [ ]:
# Comparing accuracy
plt.plot(history.history['accuracy'], label='Training accuracy', color='red')
plt.plot(history.history['val_accuracy'], label='Validation accuracy', color='blue')
plt.legend()
plt.show()